In [1]:
import numpy as np
from skimage.filters.rank import (equalize, otsu, autolevel, gradient,
                                  majority, maximum, mean, geometric_mean,
                                  subtract_mean, median, minimum, modal,
                                  enhance_contrast, pop, sum, threshold,
                                  noise_filter, entropy)
from skimage.exposure import equalize_hist
from skimage.morphology import disk, ball, cube
import skimage.util

In [2]:
b = np.load('rank_filters_tests_3d.npz')
k = b.keys() # equalize
print([i for i in k])
t_equalize = b['equalize']
t_otsu = b['otsu']
t_autolevel = b['autolevel']
t_gradient = b['gradient']
t_majority = b['majority']
t_maximum = b['maximum']
t_mean = b['mean']
t_geometric_mean = b['geometric_mean']
t_subtract_mean = b['subtract_mean']
t_median = b['median']
t_minimum = b['minimum']
t_modal = b['modal']
t_enhance_contrast = b['enhance_contrast']
t_pop = b['pop']
t_sum = b['sum']
t_threshold = b['threshold']
t_noise_filter = b['noise_filter']
t_entropy = b['entropy']
(t_equalize.shape, t_otsu.shape, t_autolevel.shape, t_gradient.shape,
t_majority.shape, t_maximum.shape, t_mean.shape, t_geometric_mean.shape,
t_subtract_mean.shape, t_median.shape, t_minimum.shape, 
t_enhance_contrast.shape, t_pop.shape, t_sum.shape, t_threshold.shape,
t_noise_filter.shape)

['equalize', 'otsu', 'autolevel', 'gradient', 'majority', 'maximum', 'mean', 'geometric_mean', 'subtract_mean', 'median', 'minimum', 'modal', 'enhance_contrast', 'pop', 'sum', 'threshold', 'noise_filter', 'entropy']


((10, 10, 10),
 (10, 10, 10),
 (10, 10, 10),
 (10, 10, 10),
 (10, 10, 10),
 (10, 10, 10),
 (10, 10, 10),
 (10, 10, 10),
 (10, 10, 10),
 (10, 10, 10),
 (10, 10, 10),
 (10, 10, 10),
 (10, 10, 10),
 (10, 10, 10),
 (10, 10, 10),
 (10, 10, 10))

# Cube Neighborhood Test

The previous implementation of local histogram equalization works only on 2D images. Local histogram equalization extracts the local neighborhood of every pixel, generates a histogram, flattens the histogram, and then uses the flattened histogram to remap the pixel to another intensity value.


In order to test the 3D implementation, I created a test_volume of random integers from 0 to 255. When the equalization algorithm is run, each voxel will be remapped based off the local histogram. To calculate the expected value based off of the working 2D implementation, I extracted a 3x3x3 neighborhood for each voxel and flattened that neighbhorhood into a 2D array. Since only the center of the neighborhood, the original voxel to be remapped, is going to change, I arranged the flattened 2D array so that the first pixel in the flattened array corresponds to the original voxel. I then ran the 2D equalize function using a neighbhorhood that was far bigger than the array. Thus, for that first pixel in that 2D array, the neighborhood used to equalize the pixel is exactly the same as the 3x3x3 neighborhood used to equalize the voxel from the volume. I then compared this calculation to my actual 3D implementation.

In [3]:
# Generates Random Volume
np.random.seed(5)
test_volume = np.random.randint(0, high = 256, size = (10,10,10), dtype = np.uint16)
correct_output = np.empty((10,10,10), dtype = np.uint16)

In [10]:
# Loops through every voxel
for i in range(10):
    for j in range(10):
        for k in range(10):
            
            pixels = []
            # Collects pixels from local 3 x 3 Cubic Neighborhood
            # S, R, C start with 0 so that first pixel to be added is the same as the voxel
            for s in [0, -1, 1]:
                for r in [0, -1, 1]:
                    for c in [0, -1, 1]:
                        ss = i + s
                        rr = j + r
                        cc = k + c
                        
                        if (ss >= 0 and rr >= 0 and cc >= 0 and ss < 10 and rr < 10 and cc < 10):
                            pixels.append(test_volume[ss][rr][cc])
            # Reshaping to a 2D numpy array
            pixels = np.asarray(pixels, dtype = np.uint8)
            pixels = pixels[np.newaxis].T
            
            # Equalizing 2D array with old implementation
#             temp = equalize(pixels, disk(50))
#             temp = otsu(pixels, disk(50))
#             temp = autolevel(pixels, disk(50))
#             temp = gradient(pixels, disk(50))
#             temp = majority(pixels, disk(50))
#             temp = maximum(pixels, disk(50))
#             temp = mean(pixels, disk(50))
#             temp = geometric_mean(pixels, disk(50))
#             temp = subtract_mean(pixels, disk(50))
#             temp = median(pixels, disk(50))
#             temp = minimum(pixels, disk(50))
#             temp = modal(pixels, disk(50))
#             temp = enhance_contrast(pixels, disk(50))
#             temp = pop(pixels, disk(50))
#             temp = sum(pixels, disk(50))
#             temp = threshold(pixels, disk(50))
#             temp = noise_filter(pixels, disk(50))
            temp = entropy(pixels, disk(50))

            # Filling in Correct Output Volume with calculated value
            correct_output[i][j][k] = temp[0][0]

[[[3 3 3 3 3 3 3 3 3 3]
  [3 4 4 4 4 4 3 4 4 3]
  [3 4 4 4 4 4 3 3 4 3]
  [3 3 3 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 3 3 3 3 3 3 3 3 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4

[[[3 3 3 3 3 3 3 3 3 3]
  [3 4 4 4 4 4 3 4 4 3]
  [3 4 4 4 4 4 3 3 4 3]
  [3 3 3 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 3 3 3 3 3 3 3 3 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4

[[[3 3 3 3 3 3 3 3 3 3]
  [3 4 4 4 4 4 3 4 4 3]
  [3 4 4 4 4 4 3 3 4 3]
  [3 3 3 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 3 3 3 3 3 3 3 3 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4

[[[3 3 3 3 3 3 3 3 3 3]
  [3 4 4 4 4 4 3 4 4 3]
  [3 4 4 4 4 4 3 3 4 3]
  [3 3 3 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 3 3 3 3 3 3 3 3 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4

[[[3 3 3 3 3 3 3 3 3 3]
  [3 4 4 4 4 4 3 4 4 3]
  [3 4 4 4 4 4 3 3 4 3]
  [3 3 3 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 3 3 3 3 3 3 3 3 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4

...

[[[3 3 3 3 3 3 3 3 3 3]
  [3 4 4 4 4 4 3 4 4 3]
  [3 4 4 4 4 4 3 3 4 3]
  [3 3 3 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 3 3 3 3 3 3 3 3 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4

[[[3 3 3 3 3 3 3 3 3 3]
  [3 4 4 4 4 4 3 4 4 3]
  [3 4 4 4 4 4 3 3 4 3]
  [3 3 3 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 3 3 3 3 3 3 3 3 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4

[[[3 3 3 3 3 3 3 3 3 3]
  [3 4 4 4 4 4 3 4 4 3]
  [3 4 4 4 4 4 3 3 4 3]
  [3 3 3 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 3 3 3 3 3 3 3 3 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4

[[[3 3 3 3 3 3 3 3 3 3]
  [3 4 4 4 4 4 3 4 4 3]
  [3 4 4 4 4 4 3 3 4 3]
  [3 3 3 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 3 3 3 3 3 3 3 3 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4

[[[3 3 3 3 3 3 3 3 3 3]
  [3 4 4 4 4 4 3 4 4 3]
  [3 4 4 4 4 4 3 3 4 3]
  [3 3 3 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]
  [3 3 3 3 3 3 3 3 3 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [3 4 4 4 4 4 4 4 4 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 4]
  [4 4 4 4 4 4 4 4 4 3]
  [4 4 4 4 4 4 4 4 4 3]
  [3 4 4 4 4 4 4 4 4 3]]

 [[3 4 4 4 4 4 4 4 4 3]
  [4 4 4

In [9]:
# Running 3D Equalization

# Sel is local neighborhood, using a 3x3x3 cube that matches for loop from earlier
sel = np.ones([3,3,3], dtype = np.uint16)
# computed_output = equalize(test_volume, sel)
# computed_output = otsu(test_volume, sel)
# computed_output = autolevel(test_volume, sel)
# computed_output = gradient(test_volume, sel)
# computed_output = majority(test_volume, sel)
# computed_output = maximum(test_volume, sel)
# computed_output = mean(test_volume, sel)
# computed_output = geometric_mean(test_volume, sel)
# computed_output = subtract_mean(test_volume, sel)
# computed_output = median(test_volume, sel)
# computed_output = minimum(test_volume, sel)
# computed_output = modal(test_volume, sel)
# computed_output = enhance_contrast(test_volume, sel)
# computed_output = pop(test_volume, sel)
# computed_output = sum(test_volume, sel)
# computed_output = threshold(test_volume, sel)
# computed_output = noise_filter(test_volume, sel)
computed_output = entropy(test_volume, sel)

[[[3.         3.5849625  3.5849625  3.5849625  3.5849625  3.5849625
   3.41829583 3.41829583 3.5849625  3.        ]
  [3.41829583 4.05881389 4.05881389 4.05881389 4.169925   4.05881389
   3.94770278 4.05881389 4.169925   3.5849625 ]
  [3.41829583 4.05881389 4.05881389 4.05881389 4.169925   4.05881389
   3.94770278 3.94770278 4.05881389 3.41829583]
  [3.41829583 3.79465347 3.94770278 4.05881389 4.169925   4.05881389
   4.05881389 4.05881389 4.05881389 3.41829583]
  [3.5849625  4.05881389 4.05881389 4.169925   4.169925   4.169925
   4.169925   4.05881389 4.05881389 3.5849625 ]
  [3.5849625  4.169925   4.169925   4.169925   4.05881389 4.05881389
   4.169925   4.05881389 4.05881389 3.5849625 ]
  [3.5849625  4.169925   4.169925   4.169925   4.169925   4.169925
   4.05881389 4.169925   4.169925   3.5849625 ]
  [3.41829583 4.05881389 4.169925   4.169925   4.169925   4.169925
   4.05881389 4.169925   4.169925   3.5849625 ]
  [3.41829583 4.05881389 4.169925   4.169925   4.169925   4.169925
   4

In [6]:
np.testing.assert_equal(correct_output, computed_output)
print("Calculated Output Matches Correct Output!")

AssertionError: 
Arrays are not equal

Mismatched elements: 992 / 1000 (99.2%)
Max absolute difference: 0.94770278
Max relative difference: 0.24006437
 x: array([[[3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
        [3, 4, 4, 4, 4, 4, 3, 4, 4, 3],
        [3, 4, 4, 4, 4, 4, 3, 3, 4, 3],...
 y: array([[[3.      , 3.584963, 3.584963, 3.584963, 3.584963, 3.584963,
         3.418296, 3.418296, 3.584963, 3.      ],
        [3.418296, 4.058814, 4.058814, 4.058814, 4.169925, 4.058814,...

# Ball Test

This is a similar test to the last one. This matches the unit test in the PyTest. A ball of radius 1 is used instead of a 3x3x3 cube. The warnings are expected by the PyTests. I kept the test this way since the previous tests in the package checked that the warnings appeared as well.

In [11]:
# Generates Random Volume
np.random.seed(0)
test_volume = np.random.rand(10,10,10)
correct_output = np.empty((10,10,10), dtype = np.uint16)

In [12]:
# Actual Expected Value

#Creates a local neighborhood that is a sphere with radius 1
neighborhood = ball(1);
for i in range(10):
    for j in range(10):
        for k in range(10):
            
            pixels = []
            #Collecting local neighborhood
            for s in [0, -1, 1]:
                for r in [0, -1, 1]:
                    for c in [0, -1, 1]:
                        if (neighborhood[s+1][r+1][c+1]):
                            ss = i + s
                            rr = j + r
                            cc = k + c

                            if (ss >= 0 and rr >= 0 and cc >= 0 and ss < 10 and rr < 10 and cc < 10):
                                pixels.append(test_volume[ss][rr][cc])
            pixels = np.asarray(pixels)
            pixels = pixels[np.newaxis].T
#             temp = equalize(pixels, disk(50))
#             temp = otsu(pixels, disk(50))
#             temp = autolevel(pixels, disk(50))
#             temp = gradient(pixels, disk(50))
#             temp = majority(pixels, disk(50))
#             temp = maximum(pixels, disk(50))
#             temp = mean(pixels, disk(50))
#             temp = geometric_mean(pixels, disk(50))
#             temp = subtract_mean(pixels, disk(50))
#             temp = median(pixels, disk(50))
#             temp = minimum(pixels, disk(50))
#             temp = modal(pixels, disk(50))
#             temp = enhance_contrast(pixels, disk(50))
#             temp = pop(pixels, disk(50))
#             temp = sum(pixels, disk(50))
#             temp = threshold(pixels, disk(50))
#             temp = noise_filter(pixels, disk(50))
            temp = entropy(pixels, disk(50))

            correct_output[i][j][k] = temp[0][0]

/Users/wangxi/opt/anaconda3/envs/skimage-dev/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3331: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filters. Convert manually using skimage.util.img_as_ubyte to silence this warning.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
np.savez_compressed('rank_filters_tests_3d.npz',
                    equalize=t_equalize,
                    otsu=t_otsu,
                    autolevel=t_autolevel,
                    gradient=t_gradient,
                    majority=t_majority,
                    maximum=t_maximum,
                    mean=t_mean,
                    geometric_mean=t_geometric_mean,
                    subtract_mean=t_subtract_mean,
                    median=t_median,
                    minimum=t_minimum,
                    modal=t_modal,
                    enhance_contrast=t_enhance_contrast,
                    pop=t_pop,
                    sum=t_sum,
                    threshold=t_threshold,
                    noise_filter=t_noise_filter,
                    entropy=correct_output)

In [14]:
# Running 3D Equalization
# computed_output = equalize(test_volume, ball(1))
# computed_output = otsu(test_volume, ball(1))
# computed_output = autolevel(test_volume, ball(1))
# computed_output = gradient(test_volume, ball(1))
# computed_output = majority(test_volume, ball(1))
# computed_output = maximum(test_volume, ball(1))
# computed_output = mean(test_volume, ball(1))
# computed_output = geometric_mean(test_volume, ball(1))
# computed_output = subtract_mean(test_volume, ball(1))
# computed_output = median(test_volume, ball(1))
# computed_output = minimum(test_volume, ball(1))
# computed_output = modal(test_volume, ball(1))
# computed_output = enhance_contrast(test_volume, ball(1))
# computed_output = pop(test_volume, ball(1))
# computed_output = sum(test_volume, ball(1))
# computed_output = threshold(test_volume, ball(1))
# computed_output = noise_filter(test_volume, ball(1))
computed_output = entropy(test_volume, ball(1))

/Users/wangxi/Desktop/scikit-image/skimage/filters/rank/generic.py:246: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filters. Convert manually using skimage.util.img_as_ubyte to silence this warning.
  image, selem, out, mask, n_bins = _handle_input_3D(image, selem, out, mask,


In [15]:
np.testing.assert_equal(correct_output, computed_output)
print("Calculated Output Matches Correct Output!")

Calculated Output Matches Correct Output!


# Proof that Z-Shift Changes Results

By default, the voxel to be changed is placed at the center of the neighborhood. Arguments can be added to change to shift this center, which should shift the results. I will use the same volume as before. The Z Shift is a new argument I added.

In [10]:
# Running 3D Equalization
computed_output = equalize(test_volume, ball(1), shift_z = 1)

In [11]:
assert not np.array_equal(correct_output, computed_output)
print("Shift-Z Changes Output!")

Shift-Z Changes Output!


In [12]:
# Calculating Expected with Cube
np.random.seed(5)
test_volume = np.random.randint(0, high = 256, size = (10,10,10), dtype = np.uint16)
correct_output = np.empty((10,10,10), dtype = np.uint16)

# Loops through every voxel
for i in range(10):
    for j in range(10):
        for k in range(10):
            
            pixels = []
            # Collects pixels from local 3 x 3 Cubic Neighborhood
            # S, R, C start with 0 so that first pixel to be added is the same as the voxel
            for s in [0, -1, 1]:
                for r in [0, -1, 1]:
                    # C changes because of z shift
                    for c in [0, -1, -2]:
                        ss = i + s
                        rr = j + r
                        cc = k + c
                        
                        if (ss >= 0 and rr >= 0 and cc >= 0 and ss < 10 and rr < 10 and cc < 10):
                            pixels.append(test_volume[ss][rr][cc])
            # Reshaping to a 2D numpy array
            pixels = np.asarray(pixels, dtype = np.uint8)
            pixels = pixels[np.newaxis].T
            
            # Equalizing 2D array with old implementation
            temp = equalize(pixels, disk(50))
            
            # Filling in Correct Output Volume with calculated value
            correct_output[i][j][k] = temp[0][0]
            
            

In [13]:
# Running 3D Equalization
sel = np.ones([3,3,3], dtype = np.uint16)
computed_output = equalize(test_volume, sel, shift_z = 1)

In [14]:
np.testing.assert_equal(correct_output, computed_output)
print("Calculated Output Matches Correct Output!")

Calculated Output Matches Correct Output!


# Comparison to 2D on Each Slice

The computed result should be different from just simply running a 2D equalizer on each slice

In [15]:
sliced = np.empty((10,10,10), dtype = np.uint16)

for i in range(10):
    sliced[i] = equalize(test_volume[i], disk(1))

output_3d = equalize(test_volume, ball(1))

In [16]:
assert not np.array_equal(sliced, output_3d)
print("Satcked 2D Output Slices and 3D Output Correctly DO NOT Match!")

Satcked 2D Output Slices and 3D Output Correctly DO NOT Match!


# Unsigned 8-bit Int vs. Float Input

The algorithm should get the same result even if the dtype of the image is different. They should be converted to 8 bit unsigned ints.

In [17]:
np.random.seed(0)
volume_uint = np.random.randint(0, high = 256, size = (10, 10, 10), dtype = np.uint8)
volume_float = skimage.util.img_as_float(volume_uint)

output_uint = equalize(volume_uint, ball(3))
output_float = equalize(volume_float, ball(3))

In [18]:
np.testing.assert_equal(output_uint, output_float)
print("Outputs are Equal!")

Outputs are Equal!


# Selem (neighborhood) and Image Dimensionality

The dimensions of the neighborhood must match the image, otherwise an error will be thrown. This is a feature I added.

In [19]:
sel = np.ones((1,1))
try:
    equalize(test_volume, sel)
except ValueError:
    print("Error Caught Correctly!")

Error Caught Correctly!
